# Solving a toy puzzle and generating puzzles with a unique solution

In this example, we will consider the following toy puzzle.

## The puzzle

You are given a list of length $k$ filled with numbers (between $0$ and $k$). For example: $[0,4,1,0]$. The puzzle is to replace the $0$'s so that (i) every number between $1$ and $k$ appears exactly once in the list, and (ii) adjacent numbers in the list differ by more than $1$. For example, a solution for the input $[0,4,1,0]$ is $[2,4,1,3]$.

## Solving the puzzle using ASP

Let's start by solving inputs for this puzzle using ASP.

In [1]:
import clingo

puzzle_input = [0,4,1,0]
k = len(puzzle_input)

asp_program = "#const k={}.\n".format(k)

# Encode the problem input
for i,number in enumerate(puzzle_input, 1):
    if number != 0:
        asp_program += "input({},{}).\n".format(i,number)

# Generate a candidate solution
asp_program += """
    cell(1..k).
    number(1..k).
    
    1 { solution(C,N) : number(N) } 1 :- cell(C).
    1 { solution(C,N) : cell(C) } 1 :- number(N).
    
    :- input(C,N), not solution(C,N).
    
    #show solution/2.
"""

# Require that adjacent cells have numbers differing by more than 1
asp_program += """
    adjacent_cell(A,B) :- cell(A), cell(B), |A-B| = 1.
    adjacent_number(N,M) :- number(N), number(M), |N-M| = 1.
    
    :- adjacent_cell(A,B), solution(A,N), solution(B,M), adjacent_number(N,M).
"""
        
# Run clingo to find answer sets, and translate them to lists
control = clingo.Control()
control.add("base", [], asp_program)
control.ground([("base", [])])
control.configuration.solve.models = 0

with control.solve(yield_=True) as handle:
    for model in handle:
        solution = [0]*k
        for atom in model.symbols(shown=True):
            if atom.name == "solution":
                i = atom.arguments[0].number - 1
                n = atom.arguments[1].number
                solution[i] = n
        print("Solution:",solution)

Solution: [2, 4, 1, 3]


## Generating puzzles with unique solutions using ASP

Next, let's see how we can generate puzzle inputs that have a unique solution using ASP. To do so, we will use the technique of [saturation](saturation.ipynb).

We begin by generating a solution for the puzzle, similarly to what we did in the first part of the example.

In [2]:
k = 4

asp_program = "#const k={}.\n".format(k)

# Generate a candidate solution
asp_program += """
    cell(1..k).
    number(1..k).
    
    1 { solution(C,N) : number(N) } 1 :- cell(C).
    1 { solution(C,N) : cell(C) } 1 :- number(N).
    
    :- input(C,N), not solution(C,N).
"""

# Require that adjacent cells have numbers differing by more than 1
asp_program += """
    adjacent_cell(A,B) :- cell(A), cell(B), |A-B| = 1.
    adjacent_number(N,M) :- number(N), number(M), |N-M| = 1.
    
    :- adjacent_cell(A,B), solution(A,N), solution(B,M), adjacent_number(N,M).
"""

Now, let's select some of the cells in the solution to hide (i.e., replace by a $0$) in the puzzle.

In [3]:
# Hide some cells
asp_program += """
    { hide(C) : cell(C) }.
    puzzle(C,N) :- solution(C,N), not hide(C).
    
    #show puzzle/2.
"""

Next, let's encode the requirement that the puzzle that we generated has a unique solution (namely the solution that we already have for it). To do this, we will use saturation to require that there may not be a different valid solution.

In [4]:
# Generate the search space for other solutions
asp_program += """
    other_solution(C,N) : number(N) :- cell(C).
"""

# State which solutions are invalid
# i.e., those that assign the same number to two different cells
asp_program += """
    invalid :- cell(A), cell(B), A != B, other_solution(A,N), other_solution(B,N).
"""

# State which solutions are incorrect
# i.e., those that do not match the puzzle that we generated,
#       those that assign neighboring cells to subsequent numbers, and
#       those that agree with the one solution that we have for our puzzle
asp_program += """
    incorrect :- puzzle(C,N), other_solution(C,M), N != M.
    incorrect :- cell(A), cell(B), other_solution(A,N), other_solution(B,M),
        adjacent_cell(A,B), adjacent_number(N,M).
    incorrect :- other_solution(C,N) : solution(C,N).
"""

# Finish our saturation
asp_program += """
    saturate :- invalid.
    saturate :- incorrect.
    other_solution(C,N) :- cell(C), number(N), saturate.
    invalid :- saturate.
    incorrect :- saturate.
    :- not saturate.
"""

Finally, let's state that we want a puzzle where only $u=1$ cells are filled in.

In [5]:
u = 1

asp_program += """
    :- not {} {{ puzzle(C,N) : cell(C), number(N) }} {}.
""".format(u,u)

This finishes our encoding. Let's call clingo to find answer sets, and translate them to lists.

In [6]:
# Run clingo to find answer sets, and translate them to lists
control = clingo.Control()
control.add("base", [], asp_program)
control.ground([("base", [])])
control.configuration.solve.models = 0

with control.solve(yield_=True) as handle:
    for model in handle:
        solution = [0]*k
        for atom in model.symbols(shown=True):
            if atom.name == "puzzle":
                i = atom.arguments[0].number - 1
                n = atom.arguments[1].number
                solution[i] = n
        print("Puzzle with a unique solution:",solution)

Puzzle with a unique solution: [2, 0, 0, 0]
Puzzle with a unique solution: [3, 0, 0, 0]
Puzzle with a unique solution: [0, 0, 0, 3]
Puzzle with a unique solution: [0, 4, 0, 0]
Puzzle with a unique solution: [0, 0, 1, 0]
Puzzle with a unique solution: [0, 0, 4, 0]
Puzzle with a unique solution: [0, 1, 0, 0]
Puzzle with a unique solution: [0, 0, 0, 2]
